In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r"C:\Users\kavir\OneDrive\Documents\GitHub\Enhancing-Public-Health-by-Identifying-Unhealthy-Foods-Promoting-Healthier-Alternatives\datasets\cleaned_final_dataset.csv")

df['healthy'] = (df['calories'] < 120).astype(int)

features = ['calories', 'protein', 'fat']
X = df[features]

y_classification = df['healthy']

y_regression = df['calories']

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X, y_classification, test_size=0.3, random_state=42)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, y_regression, test_size=0.3, random_state=42)

In [3]:
print("Classification:")
print("Train shape:", X_train_class.shape, "| Test shape:", X_test_class.shape)
print("Regression:")
print("Train shape:", X_train_reg.shape, "| Test shape:", X_test_reg.shape)

Classification:
Train shape: (853, 3) | Test shape: (366, 3)
Regression:
Train shape: (853, 3) | Test shape: (366, 3)
